# Creación del dataset para oraciones de relaciones lógicas

In [27]:
import torch
import pandas as pd

from copy import deepcopy

from utils_vocab import BasicTokenizer

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [28]:
raw_dataset = 'negacion_5.csv'
# raw_dataset = 'negacion_10.csv'
# raw_dataset = 'negacion_15.csv'
# raw_dataset = 'negacion_20.csv'

tokenizer_file = 'tokenizer_5.pkl'
# tokenizer_file = 'tokenizer_10.pkl'
# tokenizer_file = 'tokenizer_15.pkl'
# tokenizer_file = 'tokenizer_20.pkl'

csv_file_path ='bert_data_negacion_5.csv'
# csv_file_path ='bert_data_negacion_10.csv'
# csv_file_path ='bert_data_negacion_15.csv'
# csv_file_path ='bert_data_negacion_20.csv'

jabberwockie_dataset = 'negacion_jabberwockie_5.csv'
# csv_file_path ='bert_data_negacion_jabberwockie_5.csv'

## Creamos el tokenizer

In [29]:
words_df1 = pd.read_csv(raw_dataset, names=['Sentence 1', 'Sentence 2', 'Relation'])
words_jabberwockie = pd.read_csv(jabberwockie_dataset, names=['Sentence 1', 'Sentence 2', 'Relation'])
words_df = pd.concat([words_df1, words_jabberwockie], ignore_index=True)
words_df.tail(3)

,Sentence 1,Sentence 2,Relation
21297,algún jufzyl drifla o brilca,todo jufzyl drifla o brilca,0
21298,algún jufzyl drifla o brunza,todo jufzyl drifla o brunza,0
21299,algún jufzyl drifla o dernea,todo jufzyl drifla o dernea,0


In [30]:
words = list(words_df.iloc[:,0].values)
words += list(words_df.iloc[:,1].values)
print(words[:10])
print(len(words))

['todo abuelo acuerda ', 'todo abuelo aguanta ', 'todo abuelo ama ', 'todo abuelo amanece ', 'todo abuelo anochece ', 'todo actor acuerda ', 'todo actor aguanta ', 'todo actor ama ', 'todo actor amanece ', 'todo actor anochece ']
42600


In [31]:
y = [w.strip().split(' ')  for w in words]
y = [x for w in y for x in w]
y = [w for w in y if w != '']
y = list(set(y))
print(y)
print(len(y))

['algún', 'dormínico', 'abuelo', 'o', 'ningún', 'ama', 'dernea', 'blicket', 'flakle', 'amarillo', 'amplio', 'bliscea', 'anochece', 'todo', 'brispado', 'alegre', 'alto', 'y', 'aguanta', 'amanece', 'acuerda', 'brilca', 'amargo', 'flexivo', 'albañil', 'flajuf', 'jufmoq', 'drifla', 'claribundo', 'no', 'actor', 'agujero', 'jufzyl', 'alce', 'florido', 'brunza']
36


In [32]:
# Define special symbols for the tokenizer
special_symbols = ['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]']

# Create tokenizer from words

simple_tokenizer = lambda tokens_string: tokens_string.strip().split()
tokenizer = BasicTokenizer(simple_tokenizer, special_symbols)
tokenizer.initialize_from_iterable(words)

# Save to file
tokenizer.save(tokenizer_file)

In [33]:
VOCAB_SIZE = len(tokenizer.itos)
print(VOCAB_SIZE)

41


In [34]:
print(tokenizer.itos)

['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]', 'todo', 'abuelo', 'acuerda', 'aguanta', 'ama', 'amanece', 'anochece', 'actor', 'agujero', 'albañil', 'alce', 'alegre', 'alto', 'amargo', 'amplio', 'amarillo', 'y', 'o', 'no', 'algún', 'blicket', 'bliscea', 'brilca', 'brunza', 'dernea', 'drifla', 'flajuf', 'flakle', 'jufmoq', 'jufzyl', 'brispado', 'claribundo', 'dormínico', 'florido', 'flexivo', 'ningún']


## Creamos el dataset

In [35]:
sentences_df = pd.read_csv(raw_dataset, names=['sentence 1', 'sentence 2', 'relation'])
# sentences_df = pd.read_csv(jabberwockie_dataset, names=['sentence 1', 'sentence 2', 'relation'])
print(sentences_df.shape)
sentences_df.tail()

(10650, 3)


,sentence 1,sentence 2,relation
10645,algún alce amanece o anochece,todo alce amanece o anochece,0
10646,algún alce anochece o acuerda,todo alce anochece o acuerda,0
10647,algún alce anochece o aguanta,todo alce anochece o aguanta,0
10648,algún alce anochece o ama,todo alce anochece o ama,0
10649,algún alce anochece o amanece,todo alce anochece o amanece,0


In [36]:
sentences_df['tokens sentence 1'] = sentences_df['sentence 1'].apply(lambda x: tokenizer.encode(x).tokens)
sentences_df['tokens sentence 2'] = sentences_df['sentence 2'].apply(lambda x: tokenizer.encode(x).tokens)
sentences_df.head()

,sentence 1,sentence 2,relation,tokens sentence 1,tokens sentence 2
0,todo abuelo acuerda,algún abuelo no acuerda,1,"[todo, abuelo, acuerda]","[algún, abuelo, no, acuerda]"
1,todo abuelo aguanta,algún abuelo no aguanta,1,"[todo, abuelo, aguanta]","[algún, abuelo, no, aguanta]"
2,todo abuelo ama,algún abuelo no ama,1,"[todo, abuelo, ama]","[algún, abuelo, no, ama]"
3,todo abuelo amanece,algún abuelo no amanece,1,"[todo, abuelo, amanece]","[algún, abuelo, no, amanece]"
4,todo abuelo anochece,algún abuelo no anochece,1,"[todo, abuelo, anochece]","[algún, abuelo, no, anochece]"


In [37]:
def bernoulli_true_false(p):
    # Create a Bernoulli distribution with probability p
    bernoulli_dist = torch.distributions.Bernoulli(torch.tensor([p]))
    # Sample from this distribution and convert 1 to True and 0 to False
    return bernoulli_dist.sample().item() == 1

In [38]:
def Masking(token):
    # Decide whether to mask this token (20% chance)
    mask = bernoulli_true_false(0.2)

    # If mask is False, immediately return with '[PAD]' label
    if not mask:
        return token, '[PAD]'

    # If mask is True, proceed with further operations
    # Randomly decide on an operation (50% chance each)
    random_opp = bernoulli_true_false(0.5)
    random_swich = bernoulli_true_false(0.5)

    # Case 1: If mask, random_opp, and random_swich are True
    if mask and random_opp and random_swich:
        # Replace the token with '[MASK]' and set label to a random token
        mask_label = tokenizer.decode(torch.randint(0, VOCAB_SIZE, (1,)))[0]
        token_ = '[MASK]'

    # Case 2: If mask and random_opp are True, but random_swich is False
    elif mask and random_opp and not random_swich:
        # Leave the token unchanged and set label to the same token
        token_ = token
        mask_label = token

    # Case 3: If mask is True, but random_opp is False
    else:
        # Replace the token with '[MASK]' and set label to the original token
        token_ = '[MASK]'
        mask_label = token

    return token_, mask_label

In [39]:
# Test Masking
torch.manual_seed(100)
for l in range(10):
  token="abuelo"
  token_,label=Masking(token)
  if token==token_ and label=="[PAD]":
    print(token_,label,f"\t Actual token *{token}* is left unchanged")
  elif token_=="[MASK]" and label==token:
    print(token_,label,f"\t Actual token *{token}* is masked with '{token_}'")
  else:
    print(token_,label,f"\t Actual token *{token}* is replaced with random token #{label}#")

[MASK] abuelo 	 Actual token *abuelo* is masked with '[MASK]'
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
[MASK] blicket 	 Actual token *abuelo* is replaced with random token #blicket#
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged
abuelo [PAD] 	 Actual token *abuelo* is left unchanged


In [40]:
def prepare_for_mlm(tokens, include_raw_tokens=False):
    """
    Prepares tokenized text for BERT's Masked Language Model (MLM) training.

    """
    bert_input = []  # List to store sentences processed for BERT's MLM
    bert_label = []  # List to store labels for each token (mask, random, or unchanged)
    raw_tokens = []  # List to store raw tokens if needed

    for token in tokens:
        # Apply BERT's MLM masking strategy to the token
        masked_token, mask_label = Masking(token)

        # Append the processed token and its label to the current sentence and label list
        bert_input.append(masked_token)
        bert_label.append(mask_label)

        # If raw tokens are to be included, append the original token to the current raw tokens list
        if include_raw_tokens:
            raw_tokens.append(token)

    # Return the prepared lists for BERT's MLM training
    return (bert_input, bert_label, raw_tokens) if include_raw_tokens else (bert_input, bert_label)

In [41]:
torch.manual_seed(100)
# original_input="The sun sets behind the distant mountains."
original_input = "algún abuelo alegre no ama"
tokens=tokenizer.encode(original_input).tokens
bert_input, bert_label= prepare_for_mlm(tokens, include_raw_tokens=False)
print("Without raw tokens: \t ","\n \t original_input is: \t ", original_input,"\n \t bert_input is: \t ", bert_input,"\n \t bert_label is: \t ", bert_label)
print("-"*200)
torch.manual_seed(200)
bert_input, bert_label, raw_tokens_list = prepare_for_mlm(tokens, include_raw_tokens=True)
print("With raw tokens: \t ","\n \t original_input is: \t ", original_input,"\n \t bert_input is: \t ", bert_input,"\n \t bert_label is: \t ", bert_label,"\n \t raw_tokens_list is: \t ", raw_tokens_list)

Without raw tokens: 	  
 	 original_input is: 	  algún abuelo alegre no ama 
 	 bert_input is: 	  ['[MASK]', 'abuelo', 'alegre', 'no', 'ama'] 
 	 bert_label is: 	  ['algún', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
With raw tokens: 	  
 	 original_input is: 	  algún abuelo alegre no ama 
 	 bert_input is: 	  ['algún', 'abuelo', 'alegre', 'no', 'ama'] 
 	 bert_label is: 	  ['[PAD]', '[PAD]', '[PAD]', 'no', '[PAD]'] 
 	 raw_tokens_list is: 	  ['algún', 'abuelo', 'alegre', 'no', 'ama']


In [42]:
sentences_df['bert_input_label 1'] = sentences_df['tokens sentence 1'].apply(lambda x: prepare_for_mlm(x))
sentences_df['bert_input 1'] = sentences_df['bert_input_label 1'].apply(lambda x: x[0])
sentences_df['bert_label 1'] = sentences_df['bert_input_label 1'].apply(lambda x: x[1])
sentences_df['bert_input_label 2'] = sentences_df['tokens sentence 2'].apply(lambda x: prepare_for_mlm(x))
sentences_df['bert_input 2'] = sentences_df['bert_input_label 2'].apply(lambda x: x[0])
sentences_df['bert_label 2'] = sentences_df['bert_input_label 2'].apply(lambda x: x[1])
del sentences_df['bert_input_label 1']
del sentences_df['bert_input_label 2']
sentences_df.head()

,sentence 1,sentence 2,relation,tokens sentence 1,tokens sentence 2,bert_input 1,bert_label 1,bert_input 2,bert_label 2
0,todo abuelo acuerda,algún abuelo no acuerda,1,"[todo, abuelo, acuerda]","[algún, abuelo, no, acuerda]","[todo, abuelo, acuerda]","[[PAD], [PAD], [PAD]]","[algún, abuelo, no, acuerda]","[[PAD], [PAD], [PAD], [PAD]]"
1,todo abuelo aguanta,algún abuelo no aguanta,1,"[todo, abuelo, aguanta]","[algún, abuelo, no, aguanta]","[todo, abuelo, aguanta]","[[PAD], [PAD], [PAD]]","[algún, abuelo, [MASK], [MASK]]","[[PAD], [PAD], no, [MASK]]"
2,todo abuelo ama,algún abuelo no ama,1,"[todo, abuelo, ama]","[algún, abuelo, no, ama]","[todo, abuelo, [MASK]]","[[PAD], [PAD], ama]","[algún, abuelo, no, ama]","[[PAD], [PAD], [PAD], [PAD]]"
3,todo abuelo amanece,algún abuelo no amanece,1,"[todo, abuelo, amanece]","[algún, abuelo, no, amanece]","[todo, abuelo, [MASK]]","[[PAD], [PAD], amanece]","[algún, abuelo, no, amanece]","[[PAD], [PAD], no, [PAD]]"
4,todo abuelo anochece,algún abuelo no anochece,1,"[todo, abuelo, anochece]","[algún, abuelo, no, anochece]","[todo, abuelo, anochece]","[[PAD], [PAD], [PAD]]","[algún, abuelo, [MASK], anochece]","[[PAD], [PAD], no, [PAD]]"


In [43]:
input_sentences_pair = sentences_df[['bert_input 1', 'bert_input 2']].values.tolist()
input_masked_labels_pair = sentences_df[['bert_label 1', 'bert_label 2']].values.tolist()
relations = sentences_df['relation'].values.tolist()

In [44]:
def process_for_nsp(input_sentences_pair, input_masked_labels_pair, relations):
    """
    Prepares data for understanding logical relationship.

    Args:
    input_sentences (list): List of tokenized sentences.
    input_masked_labels (list): Corresponding list of masked labels for the sentences.

    Returns:
    bert_input (list): List of sentence pairs for BERT input.
    bert_label (list): List of masked labels for the sentence pairs.
    is_next (list): Binary label list where 1 indicates 'logical relationship' and 0 indicates 'not logical relationship'.
    """

    # Verify that both input lists are of the same length and have a sufficient number of sentences
    if len(input_sentences_pair) != len(input_masked_labels_pair):
        raise ValueError("Both lists, input_sentences_pair and input_masked_labels_pair, must have the same number of items.")
    if len(input_sentences_pair) != len(relations):
        raise ValueError("Both lists, input_sentences_pair and relations, must have the same number of items.")

    bert_input = []
    bert_label = []
    is_next = []

    for sentence_pair, masked_pair, relation in zip(input_sentences_pair, input_masked_labels_pair, relations):
        # append list and add  '[CLS]' and  '[SEP]' tokens
        bert_input.append([['[CLS]'] + sentence_pair[0] + ['[SEP]'], sentence_pair[1] + ['[SEP]']])
        bert_label.append([['[PAD]'] + masked_pair[0] + ['[PAD]'], masked_pair[1]+ ['[PAD]']])
        is_next.append(relation)  # Label 1 indicates these sentences have the required logical relationship

    return bert_input, bert_label, is_next

In [45]:
bert_inputs, bert_labels, is_nexts = process_for_nsp(input_sentences_pair, input_masked_labels_pair, relations)

In [46]:
bert_inputs[:10]

[[['[CLS]', 'todo', 'abuelo', 'acuerda', '[SEP]'],
  ['algún', 'abuelo', 'no', 'acuerda', '[SEP]']],
 [['[CLS]', 'todo', 'abuelo', 'aguanta', '[SEP]'],
  ['algún', 'abuelo', '[MASK]', '[MASK]', '[SEP]']],
 [['[CLS]', 'todo', 'abuelo', '[MASK]', '[SEP]'],
  ['algún', 'abuelo', 'no', 'ama', '[SEP]']],
 [['[CLS]', 'todo', 'abuelo', '[MASK]', '[SEP]'],
  ['algún', 'abuelo', 'no', 'amanece', '[SEP]']],
 [['[CLS]', 'todo', 'abuelo', 'anochece', '[SEP]'],
  ['algún', 'abuelo', '[MASK]', 'anochece', '[SEP]']],
 [['[CLS]', 'todo', 'actor', 'acuerda', '[SEP]'],
  ['[MASK]', 'actor', 'no', 'acuerda', '[SEP]']],
 [['[CLS]', 'todo', '[MASK]', 'aguanta', '[SEP]'],
  ['algún', 'actor', 'no', '[MASK]', '[SEP]']],
 [['[CLS]', '[MASK]', 'actor', 'ama', '[SEP]'],
  ['[MASK]', 'actor', 'no', 'ama', '[SEP]']],
 [['[CLS]', 'todo', 'actor', 'amanece', '[SEP]'],
  ['algún', '[MASK]', 'no', 'amanece', '[SEP]']],
 [['[CLS]', 'todo', '[MASK]', '[MASK]', '[SEP]'],
  ['algún', '[MASK]', 'no', 'anochece', '[SEP]']]

In [47]:
def prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts, to_tensor=True):
    """
    Prepare the final input lists for BERT training.
    """
    def zero_pad_list_pair(pair_, pad='[PAD]'):
        pair = deepcopy(pair_)
        max_len = max(len(pair[0]), len(pair[1]))
        #append [PAD] to each sentence in the pair till the maximum length reaches
        pair[0].extend([pad] * (max_len - len(pair[0])))
        pair[1].extend([pad] * (max_len - len(pair[1])))
        return pair[0], pair[1]

    #flatten the tensor
    flatten = lambda l: [item for sublist in l for item in sublist]
    #transform tokens to vocab indices
    tokens_to_index=lambda tokens: [tokenizer.stoi[token] for token in tokens]

    bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final = [], [], [], []

    for bert_input, bert_label,is_next in zip(bert_inputs, bert_labels,is_nexts):
        # Create segment labels for each pair of sentences
        segment_label = [[1] * len(bert_input[0]), [2] * len(bert_input[1])]

        # Zero-pad the bert_input and bert_label and segment_label
        bert_input_padded = zero_pad_list_pair(bert_input)
        bert_label_padded = zero_pad_list_pair(bert_label)
        segment_label_padded = zero_pad_list_pair(segment_label,pad=0)

        #convert to tensors
        if to_tensor:

            # Flatten the padded inputs and labels, transform tokens to their corresponding vocab indices, and convert them to tensors
            # bert_inputs_final.append(torch.tensor(tokens_to_index(flatten(bert_input_padded)),dtype=torch.int64))
            # bert_labels_final.append(torch.tensor(tokens_to_index(flatten(bert_label_padded)),dtype=torch.int64))
            # segment_labels_final.append(torch.tensor(flatten(segment_label_padded),dtype=torch.int64))
            bert_inputs_final.append(tokens_to_index(flatten(bert_input_padded)))
            bert_labels_final.append(tokens_to_index(flatten(bert_label_padded)))
            segment_labels_final.append(flatten(segment_label_padded))
            is_nexts_final.append(is_next)

        else:
          # Flatten the padded inputs and labels
            bert_inputs_final.append(flatten(bert_input_padded))
            bert_labels_final.append(flatten(bert_label_padded))
            segment_labels_final.append(flatten(segment_label_padded))
            is_nexts_final.append(is_next)

    return bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final

In [48]:
bert_inputs_final, bert_labels_final, segment_labels_final, is_nexts_final = prepare_bert_final_inputs(bert_inputs, bert_labels, is_nexts)

In [49]:
bert_inputs_final[:10]

[[2, 5, 6, 7, 3, 24, 6, 23, 7, 3],
 [2, 5, 6, 8, 3, 24, 6, 4, 4, 3],
 [2, 5, 6, 4, 3, 24, 6, 23, 9, 3],
 [2, 5, 6, 4, 3, 24, 6, 23, 10, 3],
 [2, 5, 6, 11, 3, 24, 6, 4, 11, 3],
 [2, 5, 12, 7, 3, 4, 12, 23, 7, 3],
 [2, 5, 4, 8, 3, 24, 12, 23, 4, 3],
 [2, 4, 12, 9, 3, 4, 12, 23, 9, 3],
 [2, 5, 12, 10, 3, 24, 4, 23, 10, 3],
 [2, 5, 4, 4, 3, 24, 4, 23, 11, 3]]

In [50]:
bert_labels_final[:10]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 23, 4, 1],
 [1, 1, 1, 9, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 10, 1, 1, 1, 23, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 23, 1, 1],
 [1, 1, 1, 1, 1, 24, 1, 1, 1, 1],
 [1, 1, 12, 1, 1, 1, 1, 1, 8, 1],
 [1, 5, 1, 1, 1, 24, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 12, 1, 1, 1],
 [1, 1, 12, 30, 1, 1, 12, 1, 1, 1]]

In [51]:
segment_labels_final[:10]

[[1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 2, 2, 2, 2, 2]]

In [52]:
df_final = pd.DataFrame({
    'bert_input': bert_inputs_final,
    'bert_label': bert_labels_final,
    'segment_label': segment_labels_final,
    'relation': is_nexts_final
})

df_final.to_csv(csv_file_path, index=False)

---